# Assignment 7
## Assignment 7.1
In this part of the assignment, you will partition a dataset using different strategies. You will use the `routes.parquet` dataset you created in a previous assignment. For this dataset, the key for each route will be the **three-letter source airport code** concatenated with the **three-letter destination airport code** and the **two-letter airline**. For instance, a route from Omaha Eppley Airfield (OMA) to Denver International Airport (DEN) on American Airlines (AA) has a key of OMADENAA.

In [2]:
import os
import sys
import gzip
import json
from pathlib import Path
import numpy as np
import pandas as pd
import pygeohash
import math

In [2]:
df = pd.read_parquet('data/routes.parquet')
df.head()

,airline,src_airport,dst_airport,codeshare,equipment
0,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2965.0, 'name': 'Sochi Internat...","{'airport_id': 2990.0, 'name': 'Kazan Internat...",False,[CR2]
1,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2966.0, 'name': 'Astrakhan Airp...","{'airport_id': 2990.0, 'name': 'Kazan Internat...",False,[CR2]
2,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2966.0, 'name': 'Astrakhan Airp...","{'airport_id': 2962.0, 'name': 'Mineralnyye Vo...",False,[CR2]
3,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2968.0, 'name': 'Chelyabinsk Ba...","{'airport_id': 2990.0, 'name': 'Kazan Internat...",False,[CR2]
4,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2968.0, 'name': 'Chelyabinsk Ba...","{'airport_id': 4078.0, 'name': 'Tolmachevo Air...",False,[CR2]


In [3]:
def key_generator(arr):
    try:
        src_airport = arr['src_airport']['iata'][0:3]
        dst_airport = arr['dst_airport']['iata'][0:3]
        airline = arr['airline']['iata'][0:2]
        return f'{src_airport}{dst_airport}{airline}'
    
    except:
        return "NAN"

In [4]:
df['key'] = df.apply(key_generator, axis=1)
df = df[df['key'] != "NAN"].reset_index(drop=True)
df.sample(10)

,airline,src_airport,dst_airport,codeshare,equipment,key
43359,"{'airline_id': 345, 'name': 'Air New Zealand',...","{'airport_id': 3364.0, 'name': 'Beijing Capita...","{'airport_id': 3406.0, 'name': 'Shanghai Pudon...",True,[330],PEKPVGNZ
15743,"{'airline_id': 751, 'name': 'Air China', 'alia...","{'airport_id': 3370.0, 'name': 'Guangzhou Baiy...","{'airport_id': 6346.0, 'name': 'Baotou Airport...",True,[320],CANBAVCA
65643,"{'airline_id': 220, 'name': 'Air Bourbon', 'al...","{'airport_id': 502.0, 'name': 'London Gatwick ...","{'airport_id': 4091.0, 'name': 'Madeira Airpor...",False,[321],LGWFNCZB
11048,"{'airline_id': 240, 'name': 'Air One', 'alias'...","{'airport_id': 1190.0, 'name': 'Tirana Interna...","{'airport_id': 1550.0, 'name': 'Verona Villafr...",False,[320],TIAVRNAP
36697,"{'airline_id': 3126, 'name': 'Kenya Airways', ...","{'airport_id': 1165.0, 'name': 'Kigali Interna...","{'airport_id': 4059.0, 'name': 'Jomo Kenyatta ...",False,"[E90, E70]",KGLNBOKQ
9302,"{'airline_id': 137, 'name': 'Air France', 'ali...","{'airport_id': 342.0, 'name': 'Hamburg Airport...","{'airport_id': 1273.0, 'name': 'Toulouse-Blagn...",False,[319],HAMTLSAF
22665,"{'airline_id': 837, 'name': 'Aer Lingus', 'ali...","{'airport_id': 596.0, 'name': 'Cork Airport', ...","{'airport_id': 490.0, 'name': 'Bristol Airport...",True,[AT7],ORKBRSEI
2398,"{'airline_id': 2354, 'name': 'First Air', 'ali...","{'airport_id': 55.0, 'name': 'Iqaluit Airport'...","{'airport_id': 132.0, 'name': 'Rankin Inlet Ai...",False,[73M],YFBYRT7F
63421,"{'airline_id': 4547, 'name': 'Southwest Airlin...","{'airport_id': 3720.0, 'name': 'Portland Inter...","{'airport_id': 3748.0, 'name': 'Norman Y. Mine...",False,"[73W, 733]",PDXSJCWN
53594,"{'airline_id': 17891, 'name': 'Scoot', 'alias'...","{'airport_id': 3361.0, 'name': 'Sydney Kingsfo...","{'airport_id': 3316.0, 'name': 'Singapore Chan...",False,[772],SYDSINTZ


### 7.1.a
Start by loading the dataset from the previous assignment using Pandas's read_parquet method. Next, add the concatenated key then using Panda's apply method to create a new column called key. For this part of the example, we will create 16 partitions so that we can compare it to the partitions we create from hashed keys in the next part of the assignment. The partitions are determined by the first letter of the composite key using the following partitions.


    partitions = (
        ('A', 'A'), ('B', 'B'), ('C', 'D'), ('E', 'F'),
        ('G', 'H'), ('I', 'J'), ('K', 'L'), ('M', 'M'),
        ('N', 'N'), ('O', 'P'), ('Q', 'R'), ('S', 'T'),
        ('U', 'U'), ('V', 'V'), ('W', 'X'), ('Y', 'Z')
    )
In this case ('A', 'A') means the folder should contain all of the routes whose composite key starts with A. Similarly, ('E', 'F') should contain routes whose composite key starts with E or F.

The results/kv directory should contain the following folders.


    kv
    ├── kv_key=A
    ├── kv_key=B
    ├── kv_key=C-D
    ├── kv_key=E-F
    ├── kv_key=G-H
    ├── kv_key=I-J
    ├── kv_key=K-L
    ├── kv_key=M
    ├── kv_key=N
    ├── kv_key=O-P
    ├── kv_key=Q-R
    ├── kv_key=S-T
    ├── kv_key=U
    ├── kv_key=V
    ├── kv_key=W-X
    └── kv_key=Y-Z
An easy way to create this directory structure is to create a new key called kv_key from the key column and use the to_parquet method with partition_cols=['kv_key'] to save a partitioned dataset.

In [5]:
def partition_keys(arr):
    part_dict = {"A":"A",
                 "B":"B",
                 "C":"C-D",
                 "D":"C-D",
                 "E":"E-F",
                 "F":"E-F",
                 "G":"G-H",
                 "H":"G-H",
                 "I":"I-J",
                 "J":"I-J",
                 "K":"K-L",
                 "L":"K-L",
                 "M":"M",
                 "N":"N",
                 "O":"O-P",
                 "P":"O-P",
                 "Q":"Q-R",
                 "R":"Q-R",
                 "S":"S-T",
                 "T":"S-T",
                 "U":"U",
                 "V":"V",
                 "W":"W-X",
                 "X":"W-X",
                 "Y":"Y-Z",
                 "Z":"Y-Z"}
    return(part_dict[arr[0].upper()])

In [6]:
df['kv_key'] = df['key'].apply(partition_keys)
df.sample(10)

,airline,src_airport,dst_airport,codeshare,equipment,key,kv_key
50490,"{'airline_id': 4533, 'name': 'Saudi Arabian Ai...","{'airport_id': 2076.0, 'name': 'Al Qaisumah/Ha...","{'airport_id': 2072.0, 'name': 'King Abdulaziz...",False,[E70],AQIJEDSV,A
45678,"{'airline_id': 5282, 'name': 'Ukraine Internat...","{'airport_id': 2939.0, 'name': 'Boryspil Inter...","{'airport_id': 1056.0, 'name': 'Tenerife South...",False,[737],KBPTFSPS,K-L
7235,"{'airline_id': 214, 'name': 'Air Berlin', 'ali...","{'airport_id': 347.0, 'name': 'Nuremberg Airpo...","{'airport_id': 293.0, 'name': 'Djerba Zarzis I...",False,[738],NUEDJEAB,N
24924,"{'airline_id': 1316, 'name': 'AirTran Airways'...","{'airport_id': 4011.0, 'name': 'Manchester-Bos...","{'airport_id': 3858.0, 'name': 'Minneapolis-St...",True,[73H],MHTMSPFL,M
35648,"{'airline_id': 3090, 'name': 'KLM Royal Dutch ...","{'airport_id': 580.0, 'name': 'Amsterdam Airpo...","{'airport_id': 3682.0, 'name': 'Hartsfield Jac...",False,"[, 777]",AMSATLKL,A
41936,"{'airline_id': 1758, 'name': 'China Eastern Ai...","{'airport_id': 3379.0, 'name': 'Xi'an Xianyang...","{'airport_id': 3364.0, 'name': 'Beijing Capita...",False,"[321, 320, 333]",XIYPEKMU,W-X
32331,"{'airline_id': 2822, 'name': 'Iberia Airlines'...","{'airport_id': 231.0, 'name': 'Es Senia Airpor...","{'airport_id': 1212.0, 'name': 'Alicante Inter...",True,[320],ORNALCIB,O-P
65809,"{'airline_id': 4611, 'name': 'Shenzhen Airline...","{'airport_id': 3370.0, 'name': 'Guangzhou Baiy...","{'airport_id': 3379.0, 'name': 'Xi'an Xianyang...",False,"[738, 320]",CANXIYZH,C-D
60992,"{'airline_id': 5347, 'name': 'Virgin Atlantic ...","{'airport_id': 507.0, 'name': 'London Heathrow...","{'airport_id': 3830.0, 'name': 'Chicago O'Hare...",False,[343],LHRORDVS,K-L
9851,"{'airline_id': 794, 'name': 'Air Algerie', 'al...","{'airport_id': 2177.0, 'name': 'Beirut Rafic H...","{'airport_id': 210.0, 'name': 'Houari Boumedie...",False,[736],BEYALGAH,B


In [7]:
df.to_parquet('results/kv/',partition_cols=['kv_key'])

### 7.1.b
Next, we are going to partition the dataset again, but this time we will partition by the hash value of the key. The following is a function that will create a SHA256 hash of the input key and return a hexadecimal string representation of the hash.

In [8]:
import hashlib

def hash_key(key):
    m = hashlib.sha256()
    m.update(str(key).encode('utf-8'))
    return m.hexdigest()



We will partition the data using the first character of the hexadecimal hash. As such, there are 16 possible partitions. Create a new column called hashed that is a hashed value of the key column. Next, create a partitioned dataset based on the first character of the hashed key and save the results to results/hash. The directory should contain the following folders.


    hash
    ├── hash_key=0
    ├── hash_key=1
    ├── hash_key=2
    ├── hash_key=3
    ├── hash_key=4
    ├── hash_key=5
    ├── hash_key=6
    ├── hash_key=7
    ├── hash_key=8
    ├── hash_key=9
    ├── hash_key=A
    ├── hash_key=B
    ├── hash_key=C
    ├── hash_key=D
    ├── hash_key=E
    ├── hash_key=F

In [9]:
def make_hash_key(arr):
    key = hash_key(arr['key'])
    return(key[0].upper())

In [10]:
df['hash_key'] = df.apply(make_hash_key, axis=1)

In [11]:
df.sample(10)

,airline,src_airport,dst_airport,codeshare,equipment,key,kv_key,hash_key
26283,"{'airline_id': 4296, 'name': 'Ryanair', 'alias...","{'airport_id': 535.0, 'name': 'Edinburgh Airpo...","{'airport_id': 4198.0, 'name': 'Weeze Airport'...",False,[738],EDINRNFR,E-F,D
56687,"{'airline_id': 5209, 'name': 'United Airlines'...","{'airport_id': 4046.0, 'name': 'General Wayne ...","{'airport_id': 3830.0, 'name': 'Chicago O'Hare...",True,"[ER4, ERJ, CRJ]",PIAORDUA,O-P,4
13352,"{'airline_id': 596, 'name': 'Alitalia', 'alias...","{'airport_id': 3622.0, 'name': 'Greater Roches...","{'airport_id': 3682.0, 'name': 'Hartsfield Jac...",True,[M88],ROCATLAZ,Q-R,C
25454,"{'airline_id': 4296, 'name': 'Ryanair', 'alias...","{'airport_id': 1230.0, 'name': 'Málaga Airport...","{'airport_id': 304.0, 'name': 'Brussels South ...",False,[738],AGPCRLFR,A,5
26714,"{'airline_id': 4296, 'name': 'Ryanair', 'alias...","{'airport_id': 1213.0, 'name': 'Almería Intern...","{'airport_id': 304.0, 'name': 'Brussels South ...",False,[738],LEICRLFR,K-L,B
50907,"{'airline_id': 4356, 'name': 'Sun Country Airl...","{'airport_id': 3520.0, 'name': 'Ronald Reagan ...","{'airport_id': 3544.0, 'name': 'Capital City A...",False,[73G],DCALANSY,C-D,0
5367,"{'airline_id': 24, 'name': 'American Airlines'...","{'airport_id': 3797.0, 'name': 'John F Kennedy...","{'airport_id': 2789.0, 'name': 'Jorge Chávez I...",True,[763],JFKLIMAA,I-J,0
44325,"{'airline_id': 28, 'name': 'Asiana Airlines', ...","{'airport_id': 2370.0, 'name': 'Jeju Internati...","{'airport_id': 2378.0, 'name': 'Gimpo Internat...",False,"[321, 320, 763, 333]",CJUGMPOZ,C-D,E
25637,"{'airline_id': 4296, 'name': 'Ryanair', 'alias...","{'airport_id': 1525.0, 'name': 'Il Caravaggio ...","{'airport_id': 1509.0, 'name': 'Catania-Fontan...",False,[738],BGYCTAFR,B,3
21703,"{'airline_id': 5133, 'name': 'TAAG Angola Airl...","{'airport_id': 951.0, 'name': 'Quatro de Fever...","{'airport_id': 5633.0, 'name': 'Namibe Airport...",False,[73G],LADMSZDT,K-L,0


In [12]:
df.to_parquet('results/hash/',partition_cols=['hash_key'])

### 7.1.c
Finally, we will simulate multiple geographically distributed data centers. For this example, we will assume we have three data centers located in the western, central, and eastern United States. Google lists the locations of their data centers and we will use the following locations for our three data centers.

- West
- The Dalles, Oregon
- Latitude: 45.5945645
- Longitude: -121.1786823


- Central
- Papillion, NE
- Latitude: 41.1544433
- Longitude: -96.0422378


- East
- Loudoun County, Virginia
- Latitude: 39.08344
- Longitude: -77.6497145


Assume that you have an application that provides routes for each of the source airports and you want to store routes in the data center closest to the source airport. The output folders should look as follows.

    geo
    ├── location=central
    ├── location=east
    └── location=west

In [13]:
datacenter_dict = [
    {
        "location": "west",
        "city": "The Dalles, Oregon",
        "latitude": 45.5945645,
        "longitude": -121.1786823
    },
    {
        "location": "central",
        "city": "Papillion, NE",
        "latitude": 41.1544433,
        "longitude": -96.0422378
    },
    {
        "location": "east",
        "city": "Loudoun County, Virginia",
        "latitude": 39.08344,
        "longitude": -77.6497145
    }
]

In [14]:
# Adding geohash of all the data centers to the dictionary for easy indexing
for datacenter in datacenter_dict:
    datacenter['geohash'] = pygeohash.encode(datacenter['latitude'], datacenter['longitude'])
    
datacenter_dict

[{'location': 'west',
  'city': 'The Dalles, Oregon',
  'latitude': 45.5945645,
  'longitude': -121.1786823,
  'geohash': 'c21g6s0rs4c7'},
 {'location': 'central',
  'city': 'Papillion, NE',
  'latitude': 41.1544433,
  'longitude': -96.0422378,
  'geohash': '9z7dnebnj8kb'},
 {'location': 'east',
  'city': 'Loudoun County, Virginia',
  'latitude': 39.08344,
  'longitude': -77.6497145,
  'geohash': 'dqby34cjw922'}]

In [15]:
def datacenter_search(latitude, longitude, datacenter_dict):
    # Find the geohash sourcec airport
    search_hash = pygeohash.encode(latitude, longitude)
    # Set the value of the closest data center to infinity so we can iterate down to the closest
    closest_datacenter_distance = math.inf
    # Iterate over all the data centers in the dictionary and find the closest based on geohash distance
    for datacenter in datacenter_dict:
        dist = pygeohash.geohash_haversine_distance(search_hash, datacenter['geohash'])
        if dist < closest_datacenter_distance:
            closest_datacenter = datacenter
            closest_datacenter_distance = dist
    return(closest_datacenter['location'])
    
def make_datacenter_key(row):
    try:
        lat = row.src_airport['latitude']
        long = row.src_airport['longitude']
        datacenter_key = datacenter_search(lat, long, datacenter_dict)
        return(datacenter_key)
    except:
        return "NAN"

In [16]:
df['location'] = df.apply(make_datacenter_key, axis=1)
df = df[df['location'] != "NAN"]
df

,airline,src_airport,dst_airport,codeshare,equipment,key,kv_key,hash_key,location
0,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2965.0, 'name': 'Sochi Internat...","{'airport_id': 2990.0, 'name': 'Kazan Internat...",False,[CR2],AERKZN2B,A,6,east
1,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2966.0, 'name': 'Astrakhan Airp...","{'airport_id': 2990.0, 'name': 'Kazan Internat...",False,[CR2],ASFKZN2B,A,9,east
2,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2966.0, 'name': 'Astrakhan Airp...","{'airport_id': 2962.0, 'name': 'Mineralnyye Vo...",False,[CR2],ASFMRV2B,A,1,east
3,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2968.0, 'name': 'Chelyabinsk Ba...","{'airport_id': 2990.0, 'name': 'Kazan Internat...",False,[CR2],CEKKZN2B,C-D,3,west
4,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2968.0, 'name': 'Chelyabinsk Ba...","{'airport_id': 4078.0, 'name': 'Tolmachevo Air...",False,[CR2],CEKOVB2B,C-D,1,west
...,...,...,...,...,...,...,...,...,...
66766,"{'airline_id': 4178, 'name': 'Regional Express...","{'airport_id': 6334.0, 'name': 'Whyalla Airpor...","{'airport_id': 3341.0, 'name': 'Adelaide Inter...",False,[SF3],WYAADLZL,W-X,F,west
66767,"{'airline_id': 19016, 'name': 'Apache Air', 'a...","{'airport_id': 4029.0, 'name': 'Domodedovo Int...","{'airport_id': 2912.0, 'name': 'Manas Internat...",False,[734],DMEFRUZM,C-D,8,east
66768,"{'airline_id': 19016, 'name': 'Apache Air', 'a...","{'airport_id': 2912.0, 'name': 'Manas Internat...","{'airport_id': 4029.0, 'name': 'Domodedovo Int...",False,[734],FRUDMEZM,E-F,E,west
66769,"{'airline_id': 19016, 'name': 'Apache Air', 'a...","{'airport_id': 2912.0, 'name': 'Manas Internat...","{'airport_id': 2913.0, 'name': 'Osh Airport', ...",False,[734],FRUOSSZM,E-F,8,west


In [17]:
df.to_parquet('results/geo/',partition_cols=['location'])

### 7.1.d
Create a Python function that takes as input a list of keys and the number of partitions and returns a list of keys sorted into the specified number of partitions. The partitions should be roughly equal in size. Furthermore, the partitions should have the property that each partition contains all the keys between the least key in the partition and the greatest key in the partition. In other words, the partitions should be ordered.



In [18]:
def balance_partitions(keys, partition_count):
    
    keys = sorted(keys)
    # Calculate the size of the parititions with the remainders
    partition_size, remainder = np.divmod(len(keys), partition_count) 
    
    # Create a list to add the partitions to
    partitions = []
    
    # Iterate over the number of the partitions
    for i in np.arange(partition_count):
        
        # Find the starting place to index the partition
        # If the iteration is smaller than the remainder, return the remainder instead of i
        start = i * partition_size + min(i, remainder)
        
        # Find the ending spot for the partition
        finish = (i + 1) * partition_size + min(i + 1, remainder)
        
        # Add the sorted partition to the list
        partitions.append(sorted(keys[start:finish]))
    
    # Return the partitions
    return(partitions)

In [19]:
# Generate a list of keys
keys = ['9','5','a','b','c','d','e','f','g','1','2','3']

print(f"{len(keys)} keys : {keys}")
print("")
for i in range(1,len(keys)+1):
    print(f'{str(i)+" partitions":<15}')
    for j in np.arange(0,i):
        print(f' {j+1:>3} : {balance_partitions(keys,i)[j]}', end = "\n")
    print("")

12 keys : ['9', '5', 'a', 'b', 'c', 'd', 'e', 'f', 'g', '1', '2', '3']

1 partitions   
   1 : ['1', '2', '3', '5', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g']

2 partitions   
   1 : ['1', '2', '3', '5', '9', 'a']
   2 : ['b', 'c', 'd', 'e', 'f', 'g']

3 partitions   
   1 : ['1', '2', '3', '5']
   2 : ['9', 'a', 'b', 'c']
   3 : ['d', 'e', 'f', 'g']

4 partitions   
   1 : ['1', '2', '3']
   2 : ['5', '9', 'a']
   3 : ['b', 'c', 'd']
   4 : ['e', 'f', 'g']

5 partitions   
   1 : ['1', '2', '3']
   2 : ['5', '9', 'a']
   3 : ['b', 'c']
   4 : ['d', 'e']
   5 : ['f', 'g']

6 partitions   
   1 : ['1', '2']
   2 : ['3', '5']
   3 : ['9', 'a']
   4 : ['b', 'c']
   5 : ['d', 'e']
   6 : ['f', 'g']

7 partitions   
   1 : ['1', '2']
   2 : ['3', '5']
   3 : ['9', 'a']
   4 : ['b', 'c']
   5 : ['d', 'e']
   6 : ['f']
   7 : ['g']

8 partitions   
   1 : ['1', '2']
   2 : ['3', '5']
   3 : ['9', 'a']
   4 : ['b', 'c']
   5 : ['d']
   6 : ['e']
   7 : ['f']
   8 : ['g']

9 partitions   
   1 : [